In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Load Data

In [4]:
# Read Data
import pandas as pd
df = pd.read_csv("safebear_data.csv")
df.label = df.label.replace({
    "Racisme":"identity_hate",
    "Insulte (et Discrimination)":"insult",
    "Obscénité":"obscene",
    "Sexualité explicite":"obscene",
    "Menace":"threat",
    "Toxicité":"toxic",
    "Sexisme":"toxic"
})
print("Classes \n",df.label.unique())

#['toxic','severe_toxic','obscene','threat','insult','identity_hate']
df.head(3)

Classes 
 ['identity_hate' 'insult' 'obscene' 'threat' 'toxic']


,text,label
0,De toute façon t’es qu’un sale juif t’aurais d...,identity_hate
1,Au pire retourne à la mosquée sale arabe,identity_hate
2,C’est pas l’heure d’aller bouffer des chenille...,identity_hate


# Load Model

In [5]:
MODEL_PATH = './drive/MyDrive/ML/Nuage/jigsaw/models'

In [6]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

from datasets import Dataset
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

from transformers import AutoTokenizer, AutoModelForMaskedLM

In [23]:
label_cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

ROBERTA_MODEL = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(ROBERTA_MODEL)
model = XLMRobertaForSequenceClassification.from_pretrained(MODEL_PATH+"/trained_v2", num_labels=len(label_cols), problem_type="multi_label_classification")

#model.to('cpu');

In [24]:
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

# Evaluate Data

In [25]:
def classify_text(text, return_scores=False):
  pred = pipe(text)[0]
  pred = [label for label in pred if label['score'] > 0.5]
  if return_scores:
    return pred

  return[label['label'] for label in pred]

In [26]:
# Generate Predictions
predictions = []
for i,row in df.iterrows():
    predictions.append(classify_text(row.text))
        
df['prediction'] = predictions
df['correct_multi'] = df.apply(lambda x: (x.label in x.prediction), axis=1)
df['correct_binary'] = df.apply(lambda x: (len(x.prediction) > 0), axis=1)

In [27]:
print("Multi Label:")
accuracy = df['correct_multi'].sum() / len(df) * 100
print(f"Correct Prediction / Total Item: {df['correct_multi'].sum()} / {len(df)}")
print(f"Accuracy: {accuracy:.2f}%")

print("Binary:")
accuracy = df['correct_binary'].sum() / len(df) * 100
print(f"Correct Prediction / Total Item: {df['correct_binary'].sum()} / {len(df)}")
print(f"Accuracy: {accuracy:.2f}%")


Multi Label:
Correct Prediction / Total Item: 25 / 37
Accuracy: 67.57%
Binary:
Correct Prediction / Total Item: 29 / 37
Accuracy: 78.38%


In [28]:
for i,row in df.iterrows():
    sign = "✅" if row.correct_multi else "❌"
    print(f"Text: {row.text}")
    print(f"Prediction: {row.prediction}")    
    print(f"True Label: {row.label} {sign}\n")

Text: De toute façon t’es qu’un sale juif t’aurais du mourir en 45
Prediction: ['toxic', 'obscene', 'threat', 'insult', 'identity_hate']
True Label: identity_hate ✅

Text: Au pire retourne à la mosquée sale arabe
Prediction: ['toxic', 'insult', 'identity_hate']
True Label: identity_hate ✅

Text: C’est pas l’heure d’aller bouffer des chenilles avec tes 29 femmes le négro ?
Prediction: ['toxic', 'obscene', 'insult', 'identity_hate']
True Label: identity_hate ✅

Text: Oh mon Dieu, je n’avais pas vu un Juif courir comme ça depuis les soldes du salon du cuir
Prediction: ['toxic', 'identity_hate']
True Label: identity_hate ✅

Text: Mais nan j’irai pas en enfer j’suis pas black
Prediction: ['toxic', 'identity_hate']
True Label: identity_hate ✅

Text: Pour vous les juifs le paradis c’est de trouver un cheveu dans votre soupe au resto pour vous faire rembourser
Prediction: ['toxic', 'identity_hate']
True Label: identity_hate ✅

Text: Est-ce qu'on peut se débarrasser de tous les Mexicains ?
Pred

## Custom Sentences: Neutral Performance

In [29]:
sentences = [
    "J'ai adoré la vidéo, continuez comme ça. Je suis déjà abonné.",
    "La vidéo était mauvaise. Je ne l'aime pas.",
    "Les immigrés vivent avec nous dans notre pays.",
    "Je ne suis pas d'accord avec ce que vous dites. Je pense qu'une autre solution peut être trouvée."
]
for sentence in sentences:
    label = classify_text(sentence,return_scores=True)
    print(f"Text: {sentence}")
    print(f"Prediction: {label}\n")

Text: J'ai adoré la vidéo, continuez comme ça. Je suis déjà abonné.
Prediction: []

Text: La vidéo était mauvaise. Je ne l'aime pas.
Prediction: []

Text: Les immigrés vivent avec nous dans notre pays.
Prediction: []

Text: Je ne suis pas d'accord avec ce que vous dites. Je pense qu'une autre solution peut être trouvée.
Prediction: []



## Custom Sentences: Difficult Examples

In [30]:
sentences = [
    "Je pense que les immigrés ne devraient pas exister dans notre pays.",    
    "Je ne suis pas d'accord avec votre opinion, je pense que c'est ridicule.",
    "Je ne suis pas d'accord avec ton opinion, je pense que c'est ridicule, mais je te respecte quand même.",
    "Les hommes peuvent faire des travaux plus difficiles que les femmes.",
    "La présence des femmes dans le monde du travail est importante.",
    "Les femmes ne peuvent pas réussir dans la vie des affaires.",
    "Les hommes ne peuvent pas réussir dans la vie des affaires.",
]
for sentence in sentences:
    label = classify_text(sentence, return_scores=True)
    print(f"Text: {sentence}")
    print(f"Prediction: {label}\n")

Text: Je pense que les immigrés ne devraient pas exister dans notre pays.
Prediction: [{'label': 'toxic', 'score': 0.8755549788475037}, {'label': 'identity_hate', 'score': 0.9978590607643127}]

Text: Je ne suis pas d'accord avec votre opinion, je pense que c'est ridicule.
Prediction: [{'label': 'toxic', 'score': 0.8535227179527283}, {'label': 'insult', 'score': 0.8098673820495605}]

Text: Je ne suis pas d'accord avec ton opinion, je pense que c'est ridicule, mais je te respecte quand même.
Prediction: [{'label': 'toxic', 'score': 0.8800011277198792}, {'label': 'insult', 'score': 0.8775408864021301}]

Text: Les hommes peuvent faire des travaux plus difficiles que les femmes.
Prediction: [{'label': 'identity_hate', 'score': 0.9858625531196594}]

Text: La présence des femmes dans le monde du travail est importante.
Prediction: []

Text: Les femmes ne peuvent pas réussir dans la vie des affaires.
Prediction: [{'label': 'toxic', 'score': 0.8060609698295593}, {'label': 'identity_hate', 'scor